# Nonparametric inference changes in spm1d v0.4.50  (2nd bug) (MATLAB)

(2025-07-08)

In repairing the [first bug](nonparam_bug_fix.ipynb) a second bug was discovered.

The bug affects both the Python and MATLAB versions of **spm1d** for all versions prior to v0.4.50.

The bug affects only **0D data analysis** and only two-tailed inference for the following procedures:

- spm1d.stats.nonparam.ttest
- spm1d.stats.nonparam.ttest_paired
- spm1d.stats.nonparam.ttest2
- spm1d.stats.nonparam.regress
- spm1d.stats.nonparam.ci_onesample
- spm1d.stats.nonparam.ci_pairedsample
- spm1d.stats.nonparam.ci_twosample

The bug applied a double-correction for two-tailed inference when calculating critical threshold, thereby yielding overly conservative thresholds. The p-values from these procedures are unaffected. Thus the bug affects only cases where the critical threshold is of primary concern. As such the practical implications of this bug are expected to be limited to 0D confidence interval results.

Below is a brief description of the bug and a demonstration of its resolution in spm1d v0.4.50.

<br>
<br>

___

## Preliminaries

Ensure that the spm1d source code is on your path:

In [1]:
addpath( genpath('/Users/todd/GitHub/spm1dmatlab/') )

Before starting verify that you have **spm1d** version 0.4.50 or later.

The `spm1d.version()` command may generate an error for older versions of **spm1d**.

In [2]:
disp( spm1d.version() )

0.4.50


Note that the `spm1d.stats.nonparam` subpackage has been rewritten. If you wish to replicate previous versions' results please use the `spm1d.stats.nonparam_old` package which retains the original code.

<br>
<br>

___

## Bug example

The example below shows that the old procedure a critical threshold of 2.67, considerably higher than the 2.20 and 2.23 yielded by the parametric and new nonparametric procedures.

In [3]:

rng(0)
y0         = randn(1,12);
y1         = randn(1,12);


% conduct parametric and nonparametric tests:
alpha      = 0.05;
two_tailed = true;
niter      = -1;
t          = spm1d.stats.ttest_paired(y1, y0).inference(alpha, 'two_tailed', two_tailed);
tn         = spm1d.stats.nonparam.ttest_paired(y1, y0).inference(alpha, 'two_tailed', two_tailed, 'iterations', niter);
tno        = spm1d.stats.nonparam_old.ttest_paired(y1, y0).inference(alpha, 'two_tailed', two_tailed, 'iterations', niter);

% report results
disp( sprintf( 'zstar (param):         %.5f', t.zstar) )
disp( sprintf( 'zstar (nonparam):      %.5f', tn.zstar(2)) )
disp( sprintf( 'zstar (nonparam old):  %.5f', tno.zstar(2)) )

zstar (param):         2.20099
zstar (nonparam):      2.22642
zstar (nonparam old):  2.67153


<br>

The bug can be rectified in the old version simply by multiplying the desired alpha by two:

<br>

In [4]:
tno = spm1d.stats.nonparam_old.ttest_paired(y1, y0).inference(2*alpha, 'two_tailed', two_tailed, 'iterations', niter);

disp( sprintf( 'zstar (nonparam old corrected):  %.5f', tno.zstar(2)) )

zstar (nonparam old corrected):  2.22642
